# Inference and Validation

We can use the trained network to make a prediction which is typically called **inference**. However, neural network have a tendency to perform well on training data and not generalize on unseen data. This problem is known as **overfitting**. To test the neural network for overfiting, we measure the performance on data not in the training set called the **validation** set

In [3]:
import torch
from torchvision import datasets, transforms


transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)


# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


In [9]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x))
    
        return x
        
        


In [14]:
model = Classifier()

images, label = next(iter(trainloader))
ps = torch.exp(model(images))


top_p, top_class = ps.topk(1, dim = 1)

print(top_p[:30, :])
print(top_class[:30, :])

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.1124],
        [0.1143],
        [0.1132],
        [0.1151],
        [0.1169],
        [0.1144],
        [0.1171],
        [0.1158],
        [0.1171],
        [0.1136],
        [0.1137],
        [0.1117],
        [0.1130],
        [0.1172],
        [0.1152],
        [0.1145],
        [0.1128],
        [0.1172],
        [0.1159],
        [0.1142],
        [0.1149],
        [0.1145],
        [0.1145],
        [0.1129],
        [0.1152],
        [0.1131],
        [0.1145],
        [0.1154],
        [0.1170],
        [0.1136]], grad_fn=<SliceBackward>)
tensor([[7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7],
        [7]])
